In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth 
auth.authenticate_user()


gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/train.zip" -d "/content/sample_data"



In [ ]:
!unzip '/content/drive/MyDrive/testdata.zip' -d "/content/sample_data"

In [4]:
import torch 
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import random
import cv2
import os
import math
import time
from skimage import io
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchvision.utils as vutils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.autograd import Variable
from torch.utils.data import Dataset 
from pathlib import Path
from PIL import Image
import torch.autograd as autograd
import pickle as pkl
import argparse
import logging
from tqdm import tqdm
from random import shuffle
import torch.utils.data
from torch.nn.parameter import Parameter
from torch.nn.functional import pad
from torch.nn.modules import Module
from torch.nn.modules.utils import _single, _pair, _triple
from torchvision.utils import save_image
device = torch.device("cuda")
"""root = Path('/content/sample_data/train')"""
"""labels = pd.read_csv(root/'train_label.txt', header=None, sep="\t")///"""


'labels = pd.read_csv(root/\'train_label.txt\', header=None, sep="\t")///'

In [ ]:
source_img = []
with open(os.path.join(root, 'sourcelabel.txt'), 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]
shuffle(lines)
for l in lines:
    items = l.split()
    source_img.append(items[0])

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
class CGAN(Dataset):
    def __init__(self, root_dir, labels, source, transfrom=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.labels = labels
        self.source = source
        self.transform = transfrom
    def __len__(self):
        return self.labels.shape[0] 
    def __getitem__(self, index):
        labels = self.labels.iloc[index, 1]
        img_name = os.path.join(self.root_dir, self.labels.iloc[index, 0]) 
        source = os.path.join(self.root_dir, self.source[index])
        image = io.imread(img_name)
        source_image = io.imread(source)
        image = Image.fromarray(image)
        source_image =  Image.fromarray(source_image)
        source_image227 = source_image.resize((227, 227))
        source_image128 = source_image.resize((128, 128))
        if self.transform:
            simage227 = self.transform(source_image227)
            simage128 = self.transform(source_image128)
            image = self.transform(image)
        return  simage227, simage128, image, labels
class CGANtest(Dataset):
    def __init__(self, root_dir, labels, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return self.labels.shape[0] 
    def __getitem__(self, index):
        labels = self.labels.iloc[index, 1]
        labels 
        img_name = os.path.join(self.root_dir, self.labels.iloc[index, 0])
        image = io.imread(img_name)
        pimage = Image.fromarray(image)
        if self.transform:
            image = self.transform(pimage)
        return image, labels




In [16]:
bins = [19, 29, 39, 49, 59]

def one_hot_classify(x, bins):
    x = x.to("cpu").numpy()
    idxs = np.digitize(x, bins, right=True)
    idxs = idxs.reshape(-1,1)
    idxs = torch.from_numpy(idxs)
    idxs = idxs.squeeze()
    return idxs

def one_hot_gan(x, bins):
    '''
    Convert tensor x to one-hot tensor
    '''
    x = x.to("cpu").numpy()
    idxs = np.digitize(x, bins, right=True)
    idxs = idxs.reshape(-1,1)
    z = torch.zeros(len(x), len(bins)+1).scatter_(1, torch.tensor(idxs), 1)
    return z

In [28]:
V_data = [70]
V = torch.tensor(V_data)

b = one_hot_gan(V, bins)
b.size()

torch.Size([1, 6])

In [ ]:
def real_loss(D_out, smooth=False):
    batch_size = D_out.size(0)
    # label smoothing
    if smooth:
        # smooth, real labels = 0.9
        labels = torch.ones(batch_size)*0.9
    else:
        labels = torch.ones(batch_size) # real labels = 1
    # move labels to GPU if available     
    labels = labels.to(device)
    # binary cross entropy with logits loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out, labels)  
    return loss

def fake_loss(D_out):
    batch_size = D_out.size(0)
    labels = torch.zeros(batch_size) # fake labels = 0
    labels = labels.to(device)
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out, labels)
    return loss

def accuracy(outputs, labels):
    outputs = torch.max(outputs, dim=1)[1]
    return torch.tensor(torch.sum(outputs == labels).item() / len(outputs))
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        labels = one_hot_classify(labels, bins).to(device)
        out = self(images)[0]
                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch  
        out = self(images)[0]    
       
        labels = one_hot_classify(labels, bins).to(device)
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)  # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
def check_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)


In [ ]:
class Reverse_zero_center(object):
    def __init__(self):
        pass
    def __call__(self,t_img):
        t_img=t_img/2+0.5
        return t_img

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


class ResnetClassifier(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64, pool=True)
        self.conv2 = conv_block(64, 128, pool=True)
        self.conv3 = conv_block(128, 256, pool=True)
        self.res1 = nn.Sequential(conv_block(256, 256), conv_block(256, 256))
        
        self.conv4 = conv_block(256, 512, pool=True)
        self.conv5 = conv_block(512, 1024, pool=True)
        self.res2 = nn.Sequential(conv_block(1024, 1024), conv_block(1024, 1024))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(7), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(1024, num_classes))
        
    def forward(self, xb):
        
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.res1(out) + out
        out = self.conv4(out)
        out = self.conv5(out)
        out6 = self.res2(out) + out
        out = self.classifier(out6)
    
        return [out, out6]

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
class BasicBlock(nn.Module):

    def __init__(self, inplanes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return out

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(9, 32, kernel_size=7, stride=1, padding=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 3, kernel_size=7, stride=1, padding=3)
        self.bn1 = nn.BatchNorm2d(32, eps=0.001, track_running_stats=True)
        self.bn2 = nn.BatchNorm2d(64, eps=0.001, track_running_stats=True)
        self.bn3 = nn.BatchNorm2d(128, eps=0.001, track_running_stats=True)
        self.bn4 = nn.BatchNorm2d(64, eps=0.001, track_running_stats=True)
        self.bn5 = nn.BatchNorm2d(32, eps=0.001, track_running_stats=True)
        self.repeat_blocks=self._make_repeat_blocks(BasicBlock(128,128),6)
        self.deconv1 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=0,output_padding=1)
        self.relu=nn.ReLU()
        self.tanh=nn.Tanh()

    def _make_repeat_blocks(self,block,repeat_times):
        layers=[]
        for i in range(repeat_times):
            layers.append(block)
        return nn.Sequential(*layers)

    def forward(self, x,condition=None):
        if condition is not None:
            x=torch.cat((x,condition),1)

        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.repeat_blocks(x)
        x = self.deconv1(x)
        x = self.relu(self.bn4(x))
        x = self.deconv2(x)
        x = self.relu(self.bn5(x))
        x = self.tanh(self.conv4(x))
        return x


def conv(in_channels, out_channels, kernel_size=4, stride=2, padding=1, batch_norm=True):
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
    layers.append(conv_layer)
    
    if batch_norm:
        bn = nn.BatchNorm2d(out_channels)
        layers.append(bn)
        
    return nn.Sequential(*layers)

class Discriminator(nn.Module):
    
    def __init__(self, y_size, conv_dim=64):
        '''
        Initialize the Discriminator Module
        :param conv_dim: The depth of the first convolutional layer
        :param y_size: The number of conditions 
        '''
        super(Discriminator, self).__init__()
        self.conv_dim = conv_dim
        self.y_size = y_size
        self.conv1 = conv(3, conv_dim, 4, batch_norm=False)
        self.conv2 = conv(conv_dim+y_size, conv_dim * 2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        self.conv4 = conv(conv_dim*4, conv_dim*8, 4)
        self.conv5 = conv(conv_dim*8, conv_dim*10, 4)
        self.conv6 = conv(conv_dim*10, 1, 4, 1, 0, batch_norm=False)
            
    def forward(self, x, y):
        '''
        Forward propagation of the neural network
        :param x: The input scaled image x
        :param y: One-hot encoding condition tensor y (N,y_size)
        :return: Discriminator logits; the output of the neural network
        '''
        x = F.leaky_relu(self.conv1(x))
        y = y.view(-1,y.size()[-1],1,1)
        y = y.expand(-1,-1,x.size()[-2], x.size()[-1])
        x = torch.cat([x, y], 1)
        x = F.leaky_relu(self.conv2(x))
        x = F.dropout2d(x, 0.4)
        x = F.leaky_relu(self.conv3(x))
        x = F.dropout2d(x, 0.4)
        x = F.leaky_relu(self.conv4(x))
        x = F.dropout2d(x, 0.4)
        x = F.leaky_relu(self.conv5(x))
        x = F.dropout2d(x, 0.4)
        x = self.conv6(x)
        x2 = x.squeeze()
        return x2

In [ ]:
class CGANs:
    def __init__(self,lr=0.01, gan_loss_weight=75, feature_loss_weight=0.5e-4, age_loss_weight=30):
        self.d_lr=lr
        self.g_lr=lr
        self.D = Discriminator(6, 64).to(device)
        self.G = Generator().to(device)
        self.ageclassifier = ResnetClassifier(3, 6).to(device)
        checkpoint = torch.load('/content/drive/MyDrive/pretrainresnet.pth.tar')
        epoch = checkpoint['epoch']
        print(epoch)
        self.ageclassifier.load_state_dict(checkpoint['modelstate'])
        self.ageclassifier.eval()
        self.CEcriterion = nn.CrossEntropyLoss().to(device)
        self.MSEcriterion = nn.MSELoss().to(device)
        self.gan_loss_weight=gan_loss_weight
        self.feature_loss_weight = feature_loss_weight
        self.age_loss_weight=age_loss_weight
        self.d_optimizer = optim.Adam(self.D.parameters(), self.d_lr, betas=(0.5,0.999))
        self.g_optimizer = optim.Adam(self.G.parameters(), self.g_lr, betas=(0.5,0.999))
    def save_model(self, path):
        torch.save({
            'epoch' : epoch,
            'D_state_dict': self.D.state_dict(),
            'G_state_dict': self.G.state_dict(),
            'optimizerD_state_dict': self.d_optimizer.state_dict(),
            'optimizerG_state_dict': self.g_optimizer.state_dict(),
            }, path)
    def load_generator_state_dict(self,state_dict):
        pretrained_dict = state_dict
        # step2: get model state_dict
        model_dict = self.G.state_dict()
        # step3: remove pretrained_dict params which is not in model_dict
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # step4: update model_dict using pretrained_dict
        model_dict.update(pretrained_dict)
        # step5: update model using model_dict
        self.G.load_state_dict(model_dict)
    def test_generate(self,source_img_128,condition):
        self.G.eval()
        with torch.no_grad():
            generate_image=self.G(source_img_128,condition)
        return generate_image
    def cuda(self):
        self.G = self.G.cuda()
    def train(self, source227img, source128img, true_images, age_labels):
            true_image = true_images.to(device)
            size128 = true_image.size(2)
            true_labels = one_hot_gan(age_labels, bins).to(device)
            age_labels_cf = one_hot_classify(age_labels, bins).to(device)
            true_labels128 = true_labels.view(-1,true_labels.size()[-1],1,1)
            true_labels128 = true_labels128.expand(-1,-1, size128, size128).to(device)
            flabel = age_labels + 20.
            flabel = one_hot_gan(flabel, bins)
            flabel = flabel.to(device) #need to convert to 128*128 for generator

            ###################################gan_loss###############################        
            self.G_source = self.G(source128img, condition=true_labels128)
            D_real = self.D(true_image, true_labels) #discriminator give labels
            d_real_loss = real_loss(D_real)
            D_reak_fake = self.D(true_image, flabel)
            d_realfake_loss = fake_loss(D_reak_fake)
            D_fake_real = self.D(self.G_source, true_labels)
            d_fakereal_loss = fake_loss(D_fake_real)
            d_fakereal_lossforG = real_loss(D_fake_real)

            self.d_loss=(1./2 * (d_real_loss + 1. / 2 * (d_realfake_loss + d_fakereal_loss))) * self.gan_loss_weight
            
            g_loss=(1./2*d_fakereal_lossforG)* self.gan_loss_weight

            ################################feature_loss#############################
            sourceac = self.ageclassifier(source227img)
            source_feature = sourceac[1] 
            generate_img227 = F.interpolate(self.G_source, (227, 227), mode="bilinear", align_corners=True)
            transform = torchvision.transforms.Normalize((0.5),(0.5))
            generate_img227 = transform(generate_img227)
            generateac = self.ageclassifier(generate_img227)
            generate_feature = generateac[1]
            self.feature_loss = self.MSEcriterion(source_feature, generate_feature)

             ################################age_cls_loss##############################

            age_logit = self.ageclassifier(generate_img227)[0]
            self.age_loss = self.CEcriterion(age_logit,age_labels_cf)

            self.g_loss= self.age_loss + g_loss + self.feature_loss

In [ ]:
root2 = Path('/content/sample_data/testdata')
testlabels = pd.read_csv(root2/'test_desired_age.txt', header=None, sep="\t")
finaltransfrom = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
testtransform = transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
traindata = CGAN(root, labels, source_img, transfrom=finaltransfrom)
testdata = CGANtest(root2, testlabels, transform = testtransform)
print(len(testdata))
train_set, val_set = torch.utils.data.random_split(traindata, [54999, 10000])
trainLoad = DataLoader(train_set, batch_size=80, shuffle=True, num_workers=4, pin_memory=True)
valLoad = DataLoader(val_set, batch_size=80, shuffle=False, num_workers=4, pin_memory=True)
testLoad = DataLoader(testdata, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
FtrainLoad = DeviceDataLoader(trainLoad, device)
FvalLoad = DeviceDataLoader(valLoad, device)
FtestLoad = DeviceDataLoader(testLoad, device)


5000


In [ ]:
model = CGANs(lr=1e-4, gan_loss_weight=75., feature_loss_weight=0.5e-4, age_loss_weight=30.)
                  #,feature_extractor_path=args.feature_extractor_path)
checkpoint = torch.load('/content/drive/MyDrive/GANcheckpoint2.pth.tar')
epoch = checkpoint['epoch']
print(epoch)
model.D.load_state_dict(checkpoint["D_state_dict"])

model.d_optimizer.load_state_dict(checkpoint['optimizerD_state_dict'])
model.g_optimizer.load_state_dict(checkpoint['optimizerG_state_dict'])
model.D.train()
d_optim=model.d_optimizer
g_optim=model.g_optimizer
model.G.load_state_dict(checkpoint['G_state_dict'])
model.G.eval()




In [ ]:
for test_idx, (image, labels) in enumerate(FtestLoad):
                validationpath = '/content/drive/MyDrive/testimagefr'
                save_dir = os.path.join(validationpath)
                image = Variable(image).to(device)
                labels = Variable(labels)
                aclabel = one_hot_classify(labels, bins).float().to(device)
                label = one_hot_gan(labels, bins)
                label = label.view(-1,label.size()[-1],1,1)
                labels128 = label.expand(-1,-1, 128, 128).to(device)
                img = model.test_generate(image, labels128).to(device)
                save_image(Reverse_zero_center()(img),fp=os.path.join(save_dir,"batch_%d.jpg"%test_idx))

In [ ]:
epochs = 50
for epoch in range(epochs):
      for batch_i, (simage227, simage128, image, labels) in enumerate(FtrainLoad):
            running_d_loss = None
            running_g_loss = None
            source_img_227 = Variable(simage227).to(device)
            source_img_128 = Variable(simage128).to(device)
            true_label_img = Variable(image).to(device)
            true_label = Variable(labels).to(device)

            #train discriminator
            for d_iter in range(1):
                #d_lr_scheduler.step()
                d_optim.zero_grad()
                model.train(
                    source227img = source_img_227,
                    source128img = source_img_128,
                    true_images = true_label_img,
                    age_labels = true_label
                )
                d_loss=model.d_loss
                running_d_loss=d_loss
                d_loss.backward()
                d_optim.step()

            #train generator
            for g_iter in range(2):
                #g_lr_scheduler.step()
                g_optim.zero_grad()
                model.train(
                    source227img = source_img_227,
                    source128img = source_img_128,
                    true_images = true_label_img,
                    age_labels = true_label
                )
                g_loss = model.g_loss
                running_g_loss=g_loss
                g_loss.backward()
                g_optim.step()
            if batch_i % 500 == 0:
                # append discriminator loss and generator loss
                # print discriminator and generator loss
                print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                        epoch+1, epochs, running_d_loss.item(), running_g_loss.item()))
      if (epoch+1)%10==0:
            model.save_model('/content/drive/MyDrive/GANcheckpoint2.pth.tar')
            print('got checkpoint')
            validationpath = '/content/drive/MyDrive/validation_4'
            save_dir = os.path.join(validationpath, "epoch_%d" % epoch)
            check_dir(save_dir) 
            for val_idx, (simage227, simage128, image, labels) in enumerate(FvalLoad):
                source_img_128 = Variable(simage128).to(device)
                save_image(Reverse_zero_center()(source_img_128),fp=os.path.join(save_dir,"batch_%d_source.jpg"%(val_idx)))
                agelist = [19, 29, 39, 49, 59, 69]       
                for age in range(6):
                    a = [agelist[age] for _ in range(80)]
                    a = np.array(a)
                    a = torch.from_numpy(a)
                    label = a.view(-1, 1)
                    label = one_hot_gan(label, bins)
                    label = label.view(-1,label.size()[-1],1,1)
                    labels128 = label.expand(-1,-1, 128, 128).to(device)
                    img = model.test_generate(source_img_128, labels128)
                    save_image(Reverse_zero_center()(img),fp=os.path.join(save_dir,"batch_%d_age_group_%d.jpg"%(val_idx,age)))
            print('validation image has been created!')

In [ ]:
#for evaluate
import torch 
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import random
import cv2
import os
import math
import time
from skimage import io
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchvision.utils as vutils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.autograd import Variable
from torch.utils.data import Dataset 
from pathlib import Path
from PIL import Image
import torch.autograd as autograd
import pickle as pkl
device = torch.device("cuda")

bins = [19, 29, 39, 49, 59]
def one_hot(x, bins):
    '''
    Convert tensor x to one-hot tensor
    '''
    x = x.to("cpu").numpy()
    idxs = np.digitize(x, bins, right=True)
    idxs = idxs.reshape(-1,1)
    idxs = torch.from_numpy(idxs)
    idxs = idxs.squeeze()
    return idxs
def accuracy(outputs, labels):
    outputs = torch.max(outputs, dim=1)[1]
    return torch.tensor(torch.sum(outputs == labels).item() / len(outputs))
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        labels = one_hot(labels, bins).to(device)
        out = self(images)[0]
                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch  
        out = self(images)[0]     
       
        labels = one_hot(labels, bins).to(device)
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)  # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
class CGAN(Dataset):
    def __init__(self, root_dir, labels, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return self.labels.shape[0] 
    def __getitem__(self, index):
        labels = self.labels.iloc[index, 1]
        labels 
        img_name = os.path.join(self.root_dir, self.labels.iloc[index, 0])
        image = io.imread(img_name)
        pimage = Image.fromarray(image)
        if self.transform:
            image = self.transform(pimage)
        return image, labels




def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64, pool=True)
        self.conv2 = conv_block(64, 128, pool=True)
        self.conv3 = conv_block(128, 256, pool=True)
        self.res1 = nn.Sequential(conv_block(256, 256), conv_block(256, 256))
        
        self.conv4 = conv_block(256, 512, pool=True)
        self.conv5 = conv_block(512, 1024, pool=True)
        self.res2 = nn.Sequential(conv_block(1024, 1024), conv_block(1024, 1024))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(1024, num_classes))
        
    def forward(self, xb):
        
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.res1(out) + out
        out = self.conv4(out)
        out = self.conv5(out)
        out6 = self.res2(out) + out
        out = self.classifier(out6)
    
        return [out, out6]

def fit2(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay) #weight decay
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, # learnig rata scheduler
                                                steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            if grad_clip:  # gradient clipping
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            sched.step()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        if (epoch+1)%10 == 0:
          torch.save({
             'epoch': epoch,
             'modelstate': model.state_dict(), 
          },'/content/drive/MyDrive/pretrainresneteval.pth.tar')
    return history

classifier = to_device(ResNet9(3, 6), device)

epochs = 40
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam
"""history = [evaluate(model, FvalLoad)]
history += fit2(epochs, max_lr, model, FtrainLoad, FvalLoad, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
plt.subplot(1, 2, 1)
plot_losses(history)
plt.subplot(1, 2, 2)
plot_accuracies(history)"""


'history = [evaluate(model, FvalLoad)]\nhistory += fit2(epochs, max_lr, model, FtrainLoad, FvalLoad, \n                             grad_clip=grad_clip, \n                             weight_decay=weight_decay, \n                             opt_func=opt_func)\nplt.subplot(1, 2, 1)\nplot_losses(history)\nplt.subplot(1, 2, 2)\nplot_accuracies(history)'

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/pretrainresneteval.pth.tar')
epoch = checkpoint['epoch']

print(epoch)

classifier.load_state_dict(checkpoint['modelstate'])

classifier.eval()

In [ ]:
total = torch.tensor(0.).to(device)
acclist = []
total = torch.tensor([0]).float
for test_idx, (image, labels) in enumerate(FtestLoad):
                image = Variable(image).to(device)
                labels = Variable(labels)
                aclabel = one_hot_classify(labels, bins).float().to(device)
                label = one_hot_gan(labels, bins)
                label = label.view(-1,label.size()[-1],1,1)
                labels128 = label.expand(-1,-1, 128, 128).to(device)
                img = model.test_generate(image, labels128).to(device)
                outputs = classifier(img)[0].to(device)
                acc = accuracy(outputs, aclabel)
                total+=acc

In [ ]:
total/(test_idx+1)

tensor([0.2952])

In [ ]:
#age classifier for GAN
import torch 
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import random
import cv2
import os
import math
import time
from skimage import io
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchvision.utils as vutils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.autograd import Variable
from torch.utils.data import Dataset 
from pathlib import Path
from PIL import Image
import torch.autograd as autograd
import pickle as pkl
device = torch.device("cuda")
root = Path('/content/sample_data/train')

labels = pd.read_csv(root/'train_label.txt', header=None, sep="\t")
bins = [19, 29, 39, 49, 59]
def one_hot(x, bins):
    '''
    Convert tensor x to one-hot tensor
    '''
    x = x.to("cpu").numpy()
    idxs = np.digitize(x, bins, right=True)
    idxs = idxs.reshape(-1,1)
    idxs = torch.from_numpy(idxs)
    idxs = idxs.squeeze()
    return idxs
def accuracy(outputs, labels):
    outputs = torch.max(outputs, dim=1)[1]
    return torch.tensor(torch.sum(outputs == labels).item() / len(outputs))
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        labels = one_hot(labels, bins).to(device)
        out = self(images)[0]
                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch  
        out = self(images)[0]     
       
        labels = one_hot(labels, bins).to(device)
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)  # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')


def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
class CGAN(Dataset):
    def __init__(self, root_dir, labels, transform=None) -> None:
        super().__init__()
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return self.labels.shape[0] 
    def __getitem__(self, index):
        labels = self.labels.iloc[index, 1]
        labels 
        img_name = os.path.join(self.root_dir, self.labels.iloc[index, 0])
        image = io.imread(img_name)
        pimage = Image.fromarray(image)
        if self.transform:
            image = self.transform(pimage)
        return image, labels



transform = transforms.Compose([
    transforms.Pad(4),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                         ])
testtransform = transforms.Compose([transforms.CenterCrop(128),
                   transforms.ToTensor(),
                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])
traindata = CGAN(root, labels, transform=transform)
'''testdata = CGAN(root2, testlabels, transform = testtransform)'''
train_set, val_set = torch.utils.data.random_split(traindata, [54999, 10000])
trainLoad = DataLoader(train_set, batch_size=400, shuffle=True, num_workers=4, pin_memory=True)
val_load = DataLoader(val_set, batch_size=400, shuffle=False, num_workers=4, pin_memory=True)
'''testLoad = DataLoader(testdata, batch_size=500, shuffle=False, num_workers=4, pin_memory=True)'''
FtrainLoad = DeviceDataLoader(trainLoad, device)
FvalLoad = DeviceDataLoader(val_load, device)
'''FtestLoad = DeviceDataLoader(testLoad, device)'''
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64, pool=True)
        self.conv2 = conv_block(64, 128, pool=True)
        self.conv3 = conv_block(128, 256, pool=True)
        self.res1 = nn.Sequential(conv_block(256, 256), conv_block(256, 256))
        
        self.conv4 = conv_block(256, 512, pool=True)
        self.conv5 = conv_block(512, 1024, pool=True)
        self.res2 = nn.Sequential(conv_block(1024, 1024), conv_block(1024, 1024))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(7), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(1024, num_classes))
        
    def forward(self, xb):
        
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.res1(out) + out
        out = self.conv4(out)
        out = self.conv5(out)
        out6 = self.res2(out) + out
        out = self.classifier(out6)
    
        return [out, out6]

def fit2(epochs, max_lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay) #weight decay
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, # learnig rata scheduler
                                                steps_per_epoch=len(train_loader))
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            if grad_clip:  # gradient clipping
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            optimizer.step()
            optimizer.zero_grad()
            sched.step()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        if (epoch+1)%10 == 0:
          torch.save({
             'epoch': epoch,
             'modelstate': model.state_dict(), 
          },'/content/drive/MyDrive/pretrainresnet.pth.tar')
    return history

model = to_device(ResNet9(3, 6), device)

epochs = 40
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam
"""history = [evaluate(model, FvalLoad)]
history += fit2(epochs, max_lr, model, FtrainLoad, FvalLoad, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)
plt.subplot(1, 2, 1)
plot_losses(history)
plt.subplot(1, 2, 2)
plot_accuracies(history)"""


In [ ]:

checkpoint = torch.load('/content/drive/MyDrive/pretrainresnet.pth.tar')
epoch = checkpoint['epoch']

print(epoch)

model.load_state_dict(checkpoint['modelstate'])

model.eval()

In [ ]:
history = [evaluate(model, FtestLoad)]